In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
data=pd.read_csv("/gdrive/My Drive/ML:March2020/data/online_shoppers_intention.csv")
data.head(10)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False
5,0,0.0,0,0.0,19,154.216667,0.015789,0.024561,0.0,0.0,Feb,2,2,1,3,Returning_Visitor,False,False
6,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.4,Feb,2,4,3,3,Returning_Visitor,False,False
7,1,0.0,0,0.0,0,0.000000,0.200000,0.200000,0.0,0.0,Feb,1,2,1,5,Returning_Visitor,True,False
8,0,0.0,0,0.0,2,37.000000,0.000000,0.100000,0.0,0.8,Feb,2,2,2,3,Returning_Visitor,False,False
9,0,0.0,0,0.0,3,738.000000,0.000000,0.022222,0.0,0.4,Feb,2,4,1,2,Returning_Visitor,False,False


In [6]:
data.shape

(12330, 18)

In [7]:
from sklearn.preprocessing import LabelEncoder

month_labelencoder=LabelEncoder()
data["Month"]=month_labelencoder.fit_transform(data["Month"])

visitor_labelencoder=LabelEncoder()
data["VisitorType"]=visitor_labelencoder.fit_transform(data["VisitorType"])

weekend_labelencoder=LabelEncoder()
data["Weekend"]=weekend_labelencoder.fit_transform(data["Weekend"])

revenue_labelencoder=LabelEncoder()
data["Revenue"]=revenue_labelencoder.fit_transform(data["Revenue"])

In [8]:
data.head(10)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,2,1,1,1,1,2,0,0
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.0,0.0,2,2,2,1,2,2,0,0
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,2,4,1,9,3,2,0,0
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.0,0.0,2,3,2,2,4,2,0,0
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.0,0.0,2,3,3,1,4,2,1,0
5,0,0.0,0,0.0,19,154.216667,0.015789,0.024561,0.0,0.0,2,2,2,1,3,2,0,0
6,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.4,2,2,4,3,3,2,0,0
7,1,0.0,0,0.0,0,0.000000,0.200000,0.200000,0.0,0.0,2,1,2,1,5,2,1,0
8,0,0.0,0,0.0,2,37.000000,0.000000,0.100000,0.0,0.8,2,2,2,2,3,2,0,0
9,0,0.0,0,0.0,3,738.000000,0.000000,0.022222,0.0,0.4,2,2,4,1,2,2,0,0


In [9]:
Xcol=data.drop(columns=["Revenue"])
Ycol=data["Revenue"]

In [14]:
X=np.array(Xcol)
Y=np.array(Ycol)
Y=np.reshape(Y,(-1,1))
X.shape,Y.shape

((12330, 17), (12330, 1))

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.25)

In [16]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((9247, 17), (3083, 17), (9247, 1), (3083, 1))

In [18]:
import math
def E_D(point1,point2):
  sum_squared_distance = 0
  for i in range(len(point1)):
    sum_squared_distance += math.pow(point1[i] - point2[i], 2)
  return math.sqrt(sum_squared_distance)

In [19]:
Y_train[5]

array([0])

In [20]:
Y_train[5][0]

0

In [23]:
from scipy import stats
import math
def knn(data, input_X, k, distance_fn):
  #create a list to store all predictions 
  predictions=[]   

  #for loop to make predictions for every sample in the input
  for i in range(len(input_X)):
    #store all the distances between the input and the training data
    #along with the index of the training data
    neighbor_distances_and_indices = []

    for index, train in enumerate(data): 
      #find the distance between the new input with every sample in the data
      distance = distance_fn(train[:], input_X[i])   

      #store the distance and the index wrt the training samples   
      neighbor_distances_and_indices.append((distance, index)) 

    #sort the smallest to largest distance
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    #pick the first K distances
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k] 
    
    # find the value of y in the training data for the first k distances
    k_nearest_labels = [Y_train[i][0] for distance, i in k_nearest_distances_and_indices]
    
    # find the most common value of y 
    prediction=(stats.mode(k_nearest_labels))

    #append the predicited value to the total_predictions list
    predictions.append(prediction.mode[0])
      
  return predictions # returns the list of all predictions for a given input

In [26]:
clf_query = X_test[:1]
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)
print(Y_pred)

[0]


In [27]:
clf_query = X_test
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      2613
           1       0.60      0.30      0.40       470

    accuracy                           0.86      3083
   macro avg       0.74      0.63      0.66      3083
weighted avg       0.84      0.86      0.84      3083



In [31]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, Y_pred) *100

print(' accuracy = {} %'.format(round(accuracy, 2)))

 accuracy = 86.31 %


KNN using SKlearn


In [32]:
Y_train = Y_train.ravel()

In [33]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(algorithm = 'brute',n_neighbors=4)
classifier.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [35]:
Y_pred = classifier.predict(X_test)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

[[2555   58]
 [ 358  112]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.92      2613
           1       0.66      0.24      0.35       470

    accuracy                           0.87      3083
   macro avg       0.77      0.61      0.64      3083
weighted avg       0.84      0.87      0.84      3083

